In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf

2023-07-28 10:43:35.154196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 10:43:35.307423: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-28 10:43:35.889423: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-28 10:43:35.889471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
pd.read_csv("ml-25m/tags.csv").head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [3]:
pd.read_csv("ml-25m/tags.csv").tag.unique()

array(['classic', 'sci-fi', 'dark comedy', ..., 'genre busting',
       'the wife did it', 'cornetto triolgy'], dtype=object)

In [4]:
users_rating_df = pd.read_csv("ml-25m/ratings.csv")
movies_df = pd.read_csv("ml-25m/movies.csv")

In [5]:
users_rating_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Get one hot encoded vector or genre

In [7]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
user_rating_movie_details_df = pd.merge(users_rating_df, movies_df, how="left", on="movieId")

In [9]:
user_rating_movie_details_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


Each user is garatee to have at least 20 reviews,
I have the time stamp thus I have a way to make a sequence.

In [10]:
# study for user 1
user_rating_movie_details_df.query("userId==1").sort_values(by="timestamp", ascending=True)

,userId,movieId,rating,timestamp,title,genres
36,1,5952,4.0,1147868053,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
13,1,2012,2.5,1147868068,Back to the Future Part III (1990),Adventure|Comedy|Sci-Fi|Western
12,1,2011,2.5,1147868079,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi
11,1,1653,4.0,1147868097,Gattaca (1997),Drama|Sci-Fi|Thriller
9,1,1250,4.0,1147868414,"Bridge on the River Kwai, The (1957)",Adventure|Drama|War
...,...,...,...,...,...,...
65,1,27193,3.0,1147879774,Taxi 2 (2000),Action|Comedy
32,1,5684,2.0,1147879797,Naqoyqatsi (2002),Documentary
45,1,7318,2.0,1147879850,"Passion of the Christ, The (2004)",Drama
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [11]:
# by movie

user_rating_movie_details_df.query("movieId==5952").sort_values(by=["userId", "timestamp"], ascending=True)

,userId,movieId,rating,timestamp,title,genres
36,1,5952,4.0,1147868053,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
212,2,5952,5.0,1141415528,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
453,3,5952,4.0,1439472616,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
972,4,5952,4.5,1574193472,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
2636,13,5952,4.5,1237970978,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
...,...,...,...,...,...,...
24998468,162532,5952,5.0,1418000735,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
24998762,162533,5952,5.0,1281406073,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
24999074,162534,5952,3.0,1527428422,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
24999701,162538,5952,4.0,1438781110,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy


In [12]:
user_rating_movie_details_df[["userId", "movieId"]].groupby(by="userId").count().sort_values(by="movieId", ascending=False)

,movieId
userId,
72315,32202
80974,9178
137293,8913
33844,7919
20055,7488
...,...
12094,20
119539,20
156759,20


# Build a dataset
I need to build a rating matrix Y with a row per movie and column per each user's rating.

And another R matrix that will indicate if the user has given a rating to that moview(1) or not(0)

For that to avoid having a super big matrix I will chose the first 10000 movies first

In [13]:
# get all movies ids
all_movies_ids = users_rating_df.movieId.unique()

# choose the first 5000
sampled_movies_ids = all_movies_ids[:5000]
# keep all user's rating for the fist 5000 movies
rating_from_sampled_movies_df = users_rating_df[users_rating_df.movieId.isin(sampled_movies_ids)]

#First 500 users
sampled_users_id = rating_from_sampled_movies_df.userId.unique()[:500]
rating_from_sampled_movies_df = rating_from_sampled_movies_df[users_rating_df.userId.isin(sampled_users_id)]

/tmp/ipykernel_3357/2651256587.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rating_from_sampled_movies_df = rating_from_sampled_movies_df[users_rating_df.userId.isin(sampled_users_id)]


In [14]:
rating_from_sampled_movies_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [15]:
sampled_users_id = rating_from_sampled_movies_df.userId.unique()

In [16]:
sampled_movies_ids = rating_from_sampled_movies_df.movieId.unique()

In [17]:
Nu = sampled_users_id.shape[0]
Nm = sampled_movies_ids.shape[0]

In [18]:
print(f"Users: {Nu} - Movies:{Nm}")

Users: 500 - Movies:5000


# movies features

In [19]:
all_sampled_movies = movies_df[movies_df.movieId.isin(sampled_movies_ids)]

In [20]:
all_sampled_movies["genres_array"] = all_sampled_movies.genres.str.split("|")

/tmp/ipykernel_3357/1711463164.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sampled_movies["genres_array"] = all_sampled_movies.genres.str.split("|")


In [21]:
# get all the genres
def get_all_genres(movies_df):
    genres = []

    for _, movie_row in movies_df.iterrows():
        for genre in movie_row.genres_array:
            if genre not in genres:
                genres.append(genre)

    return genres

In [22]:
genres = get_all_genres(all_sampled_movies)

In [23]:
genres

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'War',
 'Musical',
 'Documentary',
 'IMAX',
 'Western',
 'Film-Noir',
 '(no genres listed)']

In [24]:
# build dataset movies features
gender_to_categorical_dict = dict(zip(genres, range(len(genres))))

def get_one_hot_movies_genre(movies_df, gender_to_categorical_dict, genres):
    Nm = movies_df.shape[0]
    movie_index_to_id = {}
    one_hot_movies_genre = np.zeros((Nm, len(genres)), dtype="float16")
    for index, row in enumerate(movies_df.iterrows()):
        df_index, movie_row = row
        movie_index_to_id[movie_row.movieId] = index
        for genre in movie_row.genres_array:
            one_hot_movies_genre[index, gender_to_categorical_dict[genre]] = 1
    return one_hot_movies_genre, movie_index_to_id

In [25]:
movies_features, movie_dict = get_one_hot_movies_genre(all_sampled_movies, gender_to_categorical_dict, genres)

In [26]:
movies_features[0]

array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float16)

In [27]:
all_sampled_movies.iloc[0].genres

'Adventure|Animation|Children|Comedy|Fantasy'

# User features

In [28]:
user_dict = dict(zip(sampled_users_id, range(sampled_users_id.shape[0])))

In [29]:
rating_from_sampled_movies_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [30]:
user_features = np.zeros((Nu, len(genres)))

for index, row in enumerate(rating_from_sampled_movies_df.iterrows()):
    _, user_row = row
    user_index = user_dict[user_row.userId]
    movie_index = movie_dict[user_row.movieId]
    user_features[user_index] += movies_features[movie_index]

In [31]:
user_dataset = []
movie_dataset = []
Y = []

for index, row in enumerate(rating_from_sampled_movies_df.iterrows()):
    _, user_row = row
    user_index = user_dict[user_row.userId]
    movie_index = movie_dict[user_row.movieId]
    user_dataset.append(user_features[user_index])
    movie_dataset.append(movies_features[movie_index])
    Y.append(user_row.rating)

In [32]:
len(user_dataset)

59768

In [33]:
len(movie_dataset)

59768

In [34]:
user_dataset = np.array(user_dataset, dtype="float32")
movie_dataset = np.array(movie_dataset, dtype="float32")
Y = np.array(Y, dtype="float16")

# Split train, val and test set

In [35]:
# 70% for training
N = Y.shape[0]

n_train = math.ceil(N*0.7)
n_val = math.ceil(N*0.15)
n_test = N-n_train-n_val

np.random.RandomState(42).shuffle(Y)
np.random.RandomState(42).shuffle(user_dataset)
np.random.RandomState(42).shuffle(movie_dataset)

Y_train = Y[:n_train]
Y_val = Y[n_train:n_train+n_val]
Y_test = Y[n_train+n_val:]

user_train = user_dataset[:n_train]
user_val = user_dataset[n_train:n_train+n_val]
user_test = user_dataset[n_train+n_val:]

movie_train = movie_dataset[:n_train]
movie_val = movie_dataset[n_train:n_train+n_val]
movie_test = movie_dataset[n_train+n_val:]

print(f"Train N: {Y_train.shape}, Val N:{Y_val.shape}, Test:{Y_test.shape}")

Train N: (41838,), Val N:(8966,), Test:(8964,)


# Content filtering

# Scale min max

In [36]:
min = user_train.min(axis=0)
max = user_train.max(axis=0)

In [37]:
user_train-=min
user_train /=(max-min)

In [38]:
user_val-=min
user_val/=(max-min)

user_test-=min
user_test/=(max-min)

# build dataset

In [39]:
train_dataset = tf.data.Dataset.from_tensor_slices(((user_train, movie_train), Y_train)).shuffle(512).batch(256).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices(((user_val, movie_val), Y_val)).batch(128).prefetch(tf.data.AUTOTUNE)

2023-07-28 10:43:51.199995: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-28 10:43:51.240113: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-28 10:43:51.240409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-28 10:43:51.242580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# algorithm!

In [40]:
X_input_user = tf.keras.Input(shape=(20,), name="user_input")
X_input_movie = tf.keras.Input(shape=(20,), name="movie_input")

user_model = tf.keras.Sequential([tf.keras.layers.Dense(units=128, activation="relu"),
                                   tf.keras.layers.Dense(units=64, activation="relu"),
                                   tf.keras.layers.Dense(units=32, activation="relu")])

movie_model = tf.keras.Sequential([tf.keras.layers.Dense(units=128, activation="relu"),
                                   tf.keras.layers.Dense(units=64, activation="relu"),
                                   tf.keras.layers.Dense(units=32, activation="relu")])

X_user_features = user_model(X_input_user)
X_movie_features = movie_model(X_input_movie)

# concat = tf.keras.layers.Concatenate()([X_user_features, X_movie_features])
multiply = tf.keras.layers.Multiply()([X_user_features, X_movie_features])

# attention = tf.keras.layers.MultiHeadAttention(key_dim=32, num_heads=4)
# user_features_attention_on_movies_features = attention(X_user_features, X_movie_features, X_movie_features)

# proy = tf.keras.Sequential([tf.keras.layers.Dense(units=32, activation="relu"),
#                             tf.keras.layers.Dense(units=16, activation="relu"),
#                             tf.keras.layers.Dense(units=8, activation="relu")])

# concat_proy = proy(user_features_attention_on_movies_features)
# concat_proy = proy(concat)
# concat_proy = proy(multiply)

rating = tf.keras.layers.Dense(units=1)(multiply)

model = tf.keras.Model(inputs=[X_input_user, X_input_movie], outputs=[rating])

In [46]:
X_input_user = tf.keras.Input(shape=(20,), name="user_input")
X_input_movie = tf.keras.Input(shape=(20,), name="movie_input")

user_model = tf.keras.Sequential([tf.keras.layers.Dense(units=128, activation="relu"),
                                   tf.keras.layers.Dense(units=64, activation="relu"),
                                   tf.keras.layers.Dense(units=32)])

movie_model = tf.keras.Sequential([tf.keras.layers.Dense(units=128, activation="relu"),
                                   tf.keras.layers.Dense(units=64, activation="relu"),
                                   tf.keras.layers.Dense(units=32)])

X_user_features = user_model(X_input_user)
X_user_features = tf.linalg.l2_normalize(X_user_features)

X_movie_features = movie_model(X_input_movie)
X_movie_features = tf.linalg.l2_normalize(X_movie_features)

rating = tf.keras.layers.Dot(axes=(1, 1))([X_user_features, X_movie_features])

model = tf.keras.Model(inputs=[X_input_user, X_input_movie], outputs=[rating])

In [47]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 20)]         0           []                               
                                                                                                  
 movie_input (InputLayer)       [(None, 20)]         0           []                               
                                                                                                  
 sequential_10 (Sequential)     (None, 32)           13024       ['user_input[0][0]']             
                                                                                                  
 sequential_11 (Sequential)     (None, 32)           13024       ['movie_input[0][0]']            
                                                                                            

In [48]:
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=tf.keras.metrics.MeanAbsoluteError())

In [49]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="best_model_checkpoint.h5", save_best_only=True, save_weights_only=True)

In [50]:
model.fit(train_dataset, validation_data=val_dataset, epochs=200, callbacks=[early_stop, checkpoint])

Epoch 1/200
164/164 [==============================] - 2s 3ms/step - loss: 14.0825 - mean_absolute_error: 3.6013 - val_loss: 13.9589 - val_mean_absolute_error: 3.5854
Epoch 2/200
164/164 [==============================] - 0s 3ms/step - loss: 14.0742 - mean_absolute_error: 3.6002 - val_loss: 13.9525 - val_mean_absolute_error: 3.5846
Epoch 3/200
164/164 [==============================] - 0s 2ms/step - loss: 14.0726 - mean_absolute_error: 3.6000 - val_loss: 13.9511 - val_mean_absolute_error: 3.5844
Epoch 4/200
164/164 [==============================] - 0s 3ms/step - loss: 14.0722 - mean_absolute_error: 3.6000 - val_loss: 13.9506 - val_mean_absolute_error: 3.5844
Epoch 5/200
164/164 [==============================] - 0s 3ms/step - loss: 14.0720 - mean_absolute_error: 3.6000 - val_loss: 13.9503 - val_mean_absolute_error: 3.5843
Epoch 6/200
164/164 [==============================] - 0s 3ms/step - loss: 14.0719 - mean_absolute_error: 3.6000 - val_loss: 13.9501 - val_mean_absolute_error: 3.584

In [187]:
model.load_weights("best_model_checkpoint.h5")

In [188]:
rating_from_sampled_movies_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [197]:
user_id = 1
movie_id= 899

user_features_eval = np.expand_dims(user_features[user_dict[user_id]], axis=0)
movie_features_eval = np.expand_dims(movies_features[movie_dict[movie_id]], axis=0)

In [198]:
user_features_eval.shape

(1, 20)

In [199]:
user_features_eval -= min
user_features_eval /= (max-min)

In [200]:
model.predict([{"user_input": user_features_eval, "movie_input": movie_features_eval}])

1/1 [==============================] - 0s 14ms/step


array([[4.8445654]], dtype=float32)